# Reference
- https://qiita.com/ground0state/items/657861de619a4e4a30de
- https://www.kaggle.com/code/ikeppyo/jpx-lightgbm-demo

# アイディア
- 株価は時系列データなので、直近の収益率が予測値に影響されるのではないか
- 過去の2週間分（10営業日）の日次収益率（Target）から翌日の収益率を予測する
# TODO
- 日次収益率ではなく、累積収益率ではどうか？
- https://lofas.hatenablog.com/entry/2015/02/17/181406

In [ ]:
import os
from pathlib import Path
from decimal import ROUND_HALF_UP, Decimal

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [ ]:
def read_files(dir_name: str = 'train_files'):
    """
    data_specifications: データの特徴
    example_test_files: テスト用
    └sample_submission.csv: 提出用の見本
    supplemental_files: 追加されるデータ
    train_files: 訓練データ
    """
    base_path = Path(f'/kaggle/input/jpx-tokyo-stock-exchange-prediction/{dir_name}')
    prices = pd.read_csv(base_path / 'stock_prices.csv')
    options = pd.read_csv(base_path / 'options.csv')
    financials = pd.read_csv(base_path / 'financials.csv')
    trades = pd.read_csv(base_path / 'trades.csv')
    secondary_prices = pd.read_csv(base_path / 'secondary_stock_prices.csv')
    return prices, options, financials, trades, secondary_prices

In [ ]:
%%time
# 時間計測をしたい処理
stock_list = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')
train_files = read_files('train_files')
supplemental_files = read_files('supplemental_files')

In [ ]:
def merge_data(prices, options, financials, trades, secondary_prices, stock_list):
    # stock_prices がベース
    base_df = prices.copy()
    
    # stock_listと結合
    _stock_list = stock_list.copy()
    _stock_list.rename(columns={'Close': 'Close_x'}, inplace=True)
    base_df = base_df.merge(_stock_list, on='SecuritiesCode', how="left")

    # tradesと結合
    # stock_listのNewMarketSegmentと紐づくよう、tradesのSection項目を編集する
    # _trades = trades.copy()
    # _trades['NewMarketSegment'] = _trades['Section'].str.split(' \(', expand=True)[0]
    # base_df = base_df.merge(_trades, on=['Date', 'NewMarketSegment'], how="left")

    # financialsと結合
    # _financials = financials.copy()
    # _financials.rename(columns={'Date': 'Date_x', 'SecuritiesCode': 'SecuritiesCode_x'}, inplace=True)
    # base_df = base_df.merge(_financials, left_on='RowId', right_on='DateCode', how="left")
    
    return base_df

In [ ]:
def collector(prices, options, financials, trades, secondary_prices, stock_list):
    # 読み込んだデータを統合して一つのファイルに纏める
    base_df = merge_data(prices, options, financials, trades, secondary_prices, stock_list)
    return base_df

In [ ]:
%%time

base_df = collector(*train_files, stock_list)
supplemental_df = collector(*supplemental_files, stock_list)
base_df = pd.concat([base_df, supplemental_df]).reset_index(drop=True)

In [ ]:
base_df = base_df.sort_values(["SecuritiesCode", "Date"])
base_df["ReturnToday"] = base_df.groupby("SecuritiesCode")["Target"].shift(2)
for period in range(3,13):
    base_df.loc[:, f"ReturnBefore_{period-2}"] = base_df.groupby("SecuritiesCode")["Target"].shift(period)
base_df = base_df.reset_index(drop=True)
# base_df[["Date","Close","Target","ReturnToday","ReturnBefore_1","ReturnBefore_2","ReturnBefore_10"]]

In [ ]:
base_cols = ['RowId', 'Date', 'SecuritiesCode']
# 数値系の特徴量
numerical_cols = ["ReturnToday"] + [f"ReturnBefore_{period-2}" for period in range(3,13)]
# カテゴリ系の特徴量
# categorical_cols = ['NewMarketSegment', '33SectorCode']
# 目的変数
label_col = ['Target']

In [ ]:
# 予測値を降順に並べて順位番号を振る関数
# 言い換えると、目的変数から提出用項目を導出する関数
def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
# 予測用のデータフレームと、予測結果をもとに、スコアを計算する関数
def evaluator(df, pred):
    df["pred"] = pred
    df = add_rank(df)
    score = calc_spread_return_sharpe(df)
    return score

In [ ]:
import lightgbm as lgb
# import optuna.integration.lightgbm as lgb

# 学習を実行する関数
def trainer(feature_df, feat_cols, label_col, fold_params, seed=2022):
    scores = []
    models = []
    params = []

    for param in fold_params:
        ################################
        # データ準備
        ################################
        train = feature_df[(param[0] <= feature_df['Date']) & (feature_df['Date'] < param[1])]
        valid = feature_df[(param[1] <= feature_df['Date']) & (feature_df['Date'] < param[2])]

        X_train = train[feat_cols]
        y_train = train[label_col]
        X_valid = valid[feat_cols]
        y_valid = valid[label_col]
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        ################################
        # 学習
        ################################
        params = {
            'task': 'train',                   # 学習
            'boosting_type': 'gbdt',           # GBDT
            'objective': 'regression',         # 回帰
            'metric': 'rmse',                  # 損失（誤差）
            'learning_rate': 0.01,             # 学習率
            'lambda_l1': 0.5,                  # L1正則化項の係数
            'lambda_l2': 0.5,                  # L2正則化項の係数
            'num_leaves': 10,                  # 最大葉枚数
            'feature_fraction': 0.5,           # ランダムに抽出される列の割合
            'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
            'bagging_freq': 5,                 # バギング実施頻度
            'min_child_samples': 10,           # 葉に含まれる最小データ数
            'seed': seed                       # シード値
        } 
 
        lgb_results = {}                       
        model = lgb.train( 
            params,                            # ハイパーパラメータ
            lgb_train,                         # 訓練データ
            valid_sets=[lgb_train, lgb_valid], # 検証データ
            valid_names=['Train', 'Valid'],    # データセット名前
            num_boost_round=2000,              # 計算回数
            early_stopping_rounds=100,         # 計算打ち切り設定
            evals_result=lgb_results,          # 学習の履歴
            verbose_eval=100,                  # 学習過程の表示サイクル
        )  

        ################################
        # 結果描画
        ################################
        fig = plt.figure(figsize=(10, 4))

        # loss
        plt.subplot(1,2,1)
        loss_train = lgb_results['Train']['rmse']
        loss_test = lgb_results['Valid']['rmse']   
        plt.xlabel('Iteration')
        plt.ylabel('logloss')
        plt.plot(loss_train, label='train loss')
        plt.plot(loss_test, label='valid loss')
        plt.legend()

        # feature importance
        plt.subplot(1,2,2)
        importance = pd.DataFrame({'feature':feat_cols, 'importance':model.feature_importance()})
        sns.barplot(x = 'importance', y = 'feature', data = importance.sort_values('importance', ascending=False))

        plt.tight_layout()
        plt.show()

        ################################
        # 評価
        ################################
        # 推論
        pred =  model.predict(X_valid, num_iteration=model.best_iteration)
        # 評価
        score = evaluator(valid, pred)

        scores.append(score)
        models.append(model)

    print("CV_SCORES:", scores)
    print("CV_SCORE:", np.mean(scores))
    
    return models

In [ ]:
# 2020-12-23よりも前のデータは証券コードが2000個すべて揃っていないため、これ以降のデータのみを使う。
# (学習用データの開始日、学習用データの終了日＝検証用データの開始日、検証用データの終了日)
# 目的変数
label_col = ['Target']

base_cols = ['RowId', 'Date', 'SecuritiesCode']
# 数値系の特徴量
numerical_cols = ["ReturnToday"] + [f"ReturnBefore_{period-2}" for period in range(3,13)]
# 特徴量
feat_cols = numerical_cols

# データフレームの項目を選択された項目だけに絞込
feature_df = base_df[base_cols + feat_cols + label_col]

fold_params = [
    ('2020-12-23', '2021-11-01', '2021-12-01'),
    ('2021-01-23', '2021-12-01', '2022-01-01'),
    ('2021-02-23', '2022-01-01', '2022-02-01'),
]
models = trainer(feature_df, feat_cols, label_col, fold_params)

In [ ]:
def predictor(feature_df, feat_cols, models, is_train=True):
    X = feature_df[feat_cols]
    
    # 推論
    preds = list(map(lambda model: model.predict(X, num_iteration=model.best_iteration), models))
    
    # スコアは学習時のみ計算
    if is_train:
        scores = list(map(lambda pred: evaluator(feature_df, pred), preds))
        print("SCORES:", scores)

    # 推論結果をバギング
    pred = np.array(preds).mean(axis=0)

    # スコアは学習時のみ計算
    if is_train:
        score = evaluator(feature_df, pred)
        print("SCORE:", score)
    
    return pred

In [ ]:
# 試験用データは学習用にも検証用にも使用していないものを使う
test_df = feature_df[('2022-02-01' <= feature_df['Date'])].copy()
pred = predictor(test_df, feat_cols, models)

In [ ]:
# 時系列APIのロード
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
# supplemental filesを履歴データの初期状態としてセットアップ
past_df = supplemental_df.copy()
# 日次で推論・登録
for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(iter_test):
    current_date = prices["Date"].iloc[0]

    if i == 0:
        # リークを防止するため、時系列APIから受け取ったデータより未来のデータを削除
        past_df = past_df[past_df["Date"] < current_date]

    # リソース確保のため古い履歴を削除
    threshold = (pd.Timestamp(current_date) - pd.offsets.BDay(80)).strftime("%Y-%m-%d")
    past_df = past_df[past_df["Date"] >= threshold]
    
    # 時系列APIから受け取ったデータを履歴データに統合
    base_df = collector(prices, options, financials, trades, secondary_prices, stock_list)
    past_df = pd.concat([past_df, base_df]).reset_index(drop=True)

    # 特徴量エンジニアリング
    # feature_df, feat_cols, label_col = preprocessor(past_df, False)
    past_df = past_df.sort_values(["SecuritiesCode", "Date"])
    past_df["ReturnToday"] = past_df.groupby("SecuritiesCode")["Target"].shift(2)
    for period in range(3,13):
        past_df.loc[:, f"ReturnBefore_{period-2}"] = past_df.groupby("SecuritiesCode")["Target"].shift(period)
    past_df = past_df.reset_index(drop=True)
    
    label_col = ['Target']
    base_cols = ['RowId', 'Date', 'SecuritiesCode']
    numerical_cols = ["ReturnToday"] + [f"ReturnBefore_{period-2}" for period in range(3,13)]
    feat_cols = numerical_cols
    feature_df = past_df[base_cols + feat_cols + label_col]

    # 予測対象レコードだけを抽出
    feature_df = feature_df[feature_df['Date'] == current_date]

    # 推論
    feature_df["pred"] = predictor(feature_df, feat_cols, models, False)

    # 推論結果からRANKを導出し、提出データに反映
    feature_df = add_rank(feature_df)
    feature_map = feature_df.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)

    # 結果を登録
    env.predict(sample_prediction)